In [5]:
import pandas as pd
import numpy as np

In [6]:
f1 = open('channel_spend_student_data.csv')

In [7]:
lines = f1.readlines()
tiers = []
dics = []
for i in range(1, len(lines)):
    l = lines[i].replace('\n','')
    l1 = l[:5]
    l2 = eval(eval(l[6:]))
    tiers.append(l1)
    dics.append(l2)

In [8]:
print('tiers:',tiers)

tiers: ['tier1', 'tier2', 'tier3']


In [9]:
print('dics:',dics)

dics: [{'email': 1000.0, 'social': 1000.0000000000001, 'display': 1000.0, 'paid_search': 999.9999999999999, 'referral': 1000.0, 'organic_search': 0.0, 'direct': 0.0}, {'email': 2000.0, 'social': 2000.0, 'display': 2000.0, 'paid_search': 2000.0, 'referral': 2000.0, 'organic_search': 0.0, 'direct': 0.0}, {'email': 3000.0, 'social': 3000.0, 'display': 3000.0, 'paid_search': 3000.0, 'referral': 3000.0, 'organic_search': 0.0, 'direct': 0.0}]


In [10]:
df1 = pd.DataFrame(dics, index=tiers)
df1

,email,social,display,paid_search,referral,organic_search,direct
tier1,1000.0,1000.0,1000.0,1000.0,1000.0,0.0,0.0
tier2,2000.0,2000.0,2000.0,2000.0,2000.0,0.0,0.0
tier3,3000.0,3000.0,3000.0,3000.0,3000.0,0.0,0.0


In [15]:
df2 = pd.read_csv('attribution_allocation_student_data.csv')
df2

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1
1,True,referral,display,display,email,NaN,1
2,True,email,display,referral,NaN,NaN,1
3,True,referral,referral,email,NaN,NaN,1
4,True,social,referral,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...
68119,False,social,display,direct,NaN,NaN,3
68120,False,referral,referral,referral,referral,direct,3
68121,False,display,referral,social,direct,NaN,3
68122,False,email,email,social,direct,NaN,3


In [11]:
for column in df1.columns:
    print(column)

email
social
display
paid_search
referral
organic_search
direct


In [12]:
cost_df = df1.transpose()
cost_df

,tier1,tier2,tier3
email,1000.0,2000.0,3000.0
social,1000.0,2000.0,3000.0
display,1000.0,2000.0,3000.0
paid_search,1000.0,2000.0,3000.0
referral,1000.0,2000.0,3000.0
organic_search,0.0,0.0,0.0
direct,0.0,0.0,0.0


In [ ]:
# 3 allocation model

In [22]:
def count_for_weight(DataFrame, mode='first'):
    # initialize dataframe， all set to 0
    weight = {}
    columns = df1.columns.tolist()
    for tier in tiers:
        w1 = {}
        for c in columns:
            w1.update({c:0})
        weight.update({tier:w1})
        
    for index in DataFrame.index.tolist():
        s = 'tier'+str(DataFrame.iloc[index,-1])
        # filter false
        if DataFrame.iloc[index,0]:
            # first interaction
            if mode=='first':
                weight[s][DataFrame.iloc[index,1]] += 1
                
                
            # last interaction
            if mode=='last':
                nan_list = list(DataFrame.index[np.where(DataFrame.iloc[index].isna())])
                if nan_list:
                    last_loc = nan_list[0] - 1
                else:
                    last_loc = 5
                weight[s][DataFrame.iloc[index,last_loc]] += 1
                
            # linear                 
            if mode=='avg':
                nan_list = list(DataFrame.index[np.where(DataFrame.iloc[index].isna())])
                k = 5 - len(nan_list)
                for i in range(k):
                    weight[s][DataFrame.iloc[index,i+1]] += 1/k
                
    
    return weight


In [20]:
first_weight = count_for_weight(df2,mode='first')
first_df = pd.DataFrame(first_weight)
first_df

,tier1,tier2,tier3
email,264,402,618
social,475,823,982
display,435,726,870
paid_search,129,211,328
referral,1519,2719,3257
organic_search,1,7,7
direct,1,4,1


In [54]:
cost_df.div(first_df).round(2) # cac

,tier1,tier2,tier3
email,3.79,4.98,4.85
social,2.11,2.43,3.05
display,2.30,2.75,3.45
paid_search,7.75,9.48,9.15
referral,0.66,0.74,0.92
organic_search,0.00,0.00,0.00
direct,0.00,0.00,0.00


In [23]:
last_weight = count_for_weight(df2,mode='last')
last_df = pd.DataFrame(last_weight)
last_df

,tier1,tier2,tier3
email,281,555,655
social,625,1060,1275
display,528,818,946
paid_search,176,347,481
referral,1211,2107,2696
organic_search,2,4,5
direct,1,1,5


In [24]:
cost_df.div(last_df) #cac

,tier1,tier2,tier3
email,3.558719,3.603604,4.580153
social,1.600000,1.886792,2.352941
display,1.893939,2.444988,3.171247
paid_search,5.681818,5.763689,6.237006
referral,0.825764,0.949217,1.112760
organic_search,0.000000,0.000000,0.000000
direct,0.000000,0.000000,0.000000


In [52]:
avg_weight = count_for_weight(df2,mode='avg')
avg_df = pd.DataFrame(avg_weight)
avg_df.round(2)

,tier1,tier2,tier3
email,251.13,442.52,581.45
social,484.58,822.37,995.23
display,439.27,724.08,846.12
paid_search,128.05,237.82,339.00
referral,1517.10,2656.68,3290.05
organic_search,1.95,6.47,7.73
direct,1.92,2.07,3.42


In [53]:
cost_df.div(avg_df).round(2) #cac

,tier1,tier2,tier3
email,3.98,4.52,5.16
social,2.06,2.43,3.01
display,2.28,2.76,3.55
paid_search,7.81,8.41,8.85
referral,0.66,0.75,0.91
organic_search,0.00,0.00,0.00
direct,0.00,0.00,0.00


In [ ]:
# average CAC for each of the channels

In [28]:
cost_df_sum = cost_df.copy(deep=True)
cost_df_sum['total'] = cost_df_sum['tier1'] + cost_df_sum['tier2'] + cost_df_sum['tier3']
cost_df_sum

,tier1,tier2,tier3,total
email,1000.0,2000.0,3000.0,6000.0
social,1000.0,2000.0,3000.0,6000.0
display,1000.0,2000.0,3000.0,6000.0
paid_search,1000.0,2000.0,3000.0,6000.0
referral,1000.0,2000.0,3000.0,6000.0
organic_search,0.0,0.0,0.0,0.0
direct,0.0,0.0,0.0,0.0


In [55]:
avg_weight_sum = avg_df.copy(deep=True)
avg_weight_sum['total'] = avg_weight_sum['tier1'] + avg_weight_sum['tier2'] + avg_weight_sum['tier3']
avg_weight_sum.round(2)

,tier1,tier2,tier3,total
email,251.13,442.52,581.45,1275.10
social,484.58,822.37,995.23,2302.18
display,439.27,724.08,846.12,2009.47
paid_search,128.05,237.82,339.00,704.87
referral,1517.10,2656.68,3290.05,7463.83
organic_search,1.95,6.47,7.73,16.15
direct,1.92,2.07,3.42,7.40


In [42]:
first_weight_sum = first_df.copy(deep=True)
first_weight_sum['total']=first_weight_sum['tier1']+first_weight_sum['tier2']+first_weight_sum['tier3']
first_weight_sum

,tier1,tier2,tier3,total
email,264,402,618,1284
social,475,823,982,2280
display,435,726,870,2031
paid_search,129,211,328,668
referral,1519,2719,3257,7495
organic_search,1,7,7,15
direct,1,4,1,6


In [45]:
last_weight_sum = last_df.copy(deep=True)
last_weight_sum['total']=last_weight_sum['tier1']+last_weight_sum['tier2']+last_weight_sum['tier3']
last_weight_sum

,tier1,tier2,tier3,total
email,281,555,655,1491
social,625,1060,1275,2960
display,528,818,946,2292
paid_search,176,347,481,1004
referral,1211,2107,2696,6014
organic_search,2,4,5,11
direct,1,1,5,7


In [56]:
avg_df_sum = cost_df_sum.div(avg_weight_sum)
avg_df_sum.round(2)

,tier1,tier2,tier3,total
email,3.98,4.52,5.16,4.71
social,2.06,2.43,3.01,2.61
display,2.28,2.76,3.55,2.99
paid_search,7.81,8.41,8.85,8.51
referral,0.66,0.75,0.91,0.80
organic_search,0.00,0.00,0.00,0.00
direct,0.00,0.00,0.00,0.00


In [57]:
first_df_sum = cost_df_sum.div(first_weight_sum)
first_df_sum.round(2)

,tier1,tier2,tier3,total
email,3.79,4.98,4.85,4.67
social,2.11,2.43,3.05,2.63
display,2.30,2.75,3.45,2.95
paid_search,7.75,9.48,9.15,8.98
referral,0.66,0.74,0.92,0.80
organic_search,0.00,0.00,0.00,0.00
direct,0.00,0.00,0.00,0.00


In [58]:
last_df_sum = cost_df_sum.div(last_weight_sum)
last_df_sum.round(2)

,tier1,tier2,tier3,total
email,3.56,3.60,4.58,4.02
social,1.60,1.89,2.35,2.03
display,1.89,2.44,3.17,2.62
paid_search,5.68,5.76,6.24,5.98
referral,0.83,0.95,1.11,1.00
organic_search,0.00,0.00,0.00,0.00
direct,0.00,0.00,0.00,0.00


In [48]:
# mcac

In [49]:
cost_df_margin = cost_df.copy(deep=True)
cost_df_margin['tier3'] = cost_df_margin['tier3'] - cost_df_margin['tier2']
cost_df_margin['tier2'] = cost_df_margin['tier2'] - cost_df_margin['tier1']
cost_df_margin

,tier1,tier2,tier3
email,1000.0,1000.0,1000.0
social,1000.0,1000.0,1000.0
display,1000.0,1000.0,1000.0
paid_search,1000.0,1000.0,1000.0
referral,1000.0,1000.0,1000.0
organic_search,0.0,0.0,0.0
direct,0.0,0.0,0.0


In [50]:
avg_weight_margin = avg_df.copy(deep=True)
avg_weight_margin['tier3'] = avg_weight_margin['tier3'] - avg_weight_margin['tier2'] 
avg_weight_margin['tier2'] = avg_weight_margin['tier2'] - avg_weight_margin['tier1'] 
avg_weight_margin.round(2)

,tier1,tier2,tier3
email,251.13,191.38,138.93
social,484.58,337.78,172.87
display,439.27,284.82,122.03
paid_search,128.05,109.77,101.18
referral,1517.10,1139.58,633.37
organic_search,1.95,4.52,1.27
direct,1.92,0.15,1.35


In [51]:
avg_cac_margin = cost_df_margin.div(avg_weight_margin)
avg_cac_margin.round(2)

,tier1,tier2,tier3
email,3.98,5.23,7.20
social,2.06,2.96,5.78
display,2.28,3.51,8.19
paid_search,7.81,9.11,9.88
referral,0.66,0.88,1.58
organic_search,0.00,0.00,0.00
direct,0.00,0.00,0.00
